In [2]:
!pip install transformer_lens torch plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.1 MB/s eta 0:00:00
  Created wheel for transformers-stream-generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=f4b14826ad044fc39acf01cdcfb245b887844a673680685dd0e3d2fa5fe273ce
  Stored in directory: /root/.cache/pip/wheels/a8/58/d2/014cb67c3cc6def738c1b1635dbf4e3dab6fb63aba7070dce0
Successfully built transformers-stream-generator
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: beartype
    Found existing install

In [1]:
import torch
import transformer_lens as tl
import plotly.express as px
import plotly.graph_objects as go


model=tl.HookedTransformer.from_pretrained("gpt2-small")
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


using a library called **TransformerLens**
 because standard PyTorch models are "black boxes",they give you the output, but hide the internal numbers. **TransformerLens** lets us hook into the middle layers

In [2]:
example_text="""Large Language Models learn patterns from vast oceans of text, predicting meaning one token at a time.
You have the right to action, but never to the action...."""

logits,cache=model.run_with_cache(example_text)

logits.shape

torch.Size([1, 34, 50257])

torch.Size([1, 36, 50257])
* 1=batch size
* 36=sequence length
* 50257: Vocabulary size (The model calculated the probability for every single word in its dictionary to be the next word).

*logits= predictions for the next word

* blocks.0...: Layer 0 (Bottom).

* mlp: The feed-forward network.

* hook_post: The activation values after the GeLU function.

In [3]:
for layer in range(0,12):
    print(f"Layer {layer}: {cache[f'blocks.{layer}.mlp.hook_post'].shape}")

Layer 0: torch.Size([1, 34, 3072])
Layer 1: torch.Size([1, 34, 3072])
Layer 2: torch.Size([1, 34, 3072])
Layer 3: torch.Size([1, 34, 3072])
Layer 4: torch.Size([1, 34, 3072])
Layer 5: torch.Size([1, 34, 3072])
Layer 6: torch.Size([1, 34, 3072])
Layer 7: torch.Size([1, 34, 3072])
Layer 8: torch.Size([1, 34, 3072])
Layer 9: torch.Size([1, 34, 3072])
Layer 10: torch.Size([1, 34, 3072])
Layer 11: torch.Size([1, 34, 3072])


This loop iterates from `layer` 0 to 11 and prints the shape of the activations after the MLP in each block. The output `[1, 34, 3072]` indicates:
* `1`: Batch size
* `34`: Sequence length
* `3072`: The dimension of the MLP output for a single token (model's hidden dimension for GPT2-small is 768, and MLP is typically 4 times this, so 768 * 4 = 3072).

In [4]:
print(cache[f'blocks.{layer}.mlp.hook_post'].shape)

torch.Size([1, 34, 3072])


In [5]:
acts=cache['blocks.2.mlp.hook_post'][0]
values,indices=torch.max(acts,dim=0)
values.shape


torch.Size([3072])

In [6]:
top_neuron_index=torch.argmax(values)
print(top_neuron_index)

tensor(666, device='cuda:0')


In [7]:
list_of_text=model.to_str_tokens(example_text)
neuron_activations=acts[:,1612]
print(list_of_text)
print(neuron_activations)

['<|endoftext|>', 'Large', ' Language', ' Models', ' learn', ' patterns', ' from', ' vast', ' oceans', ' of', ' text', ',', ' predicting', ' meaning', ' one', ' token', ' at', ' a', ' time', '.', '\n', 'You', ' have', ' the', ' right', ' to', ' action', ',', ' but', ' never', ' to', ' the', ' action', '....']
tensor([-0.0412, -0.1404, -0.0018, -0.1458, -0.0041, -0.1281, -0.0706, -0.0411,
        -0.0920, -0.0738, -0.0997, -0.0179, -0.0790, -0.0658, -0.0305, -0.0394,
        -0.0445, -0.0231, -0.1092, -0.0728, -0.0407, -0.0840, -0.0258, -0.0684,
        -0.0608, -0.1017, -0.0325, -0.0132, -0.0094, -0.0199, -0.0386, -0.0939,
        -0.0345, -0.0707], device='cuda:0')


In [8]:
import plotly.express as px

# We convert the tensor to a standard Python list for plotting
y_values = neuron_activations.tolist()

# Create the Bar Chart
fig = px.line(
    x=list_of_text,
    y=y_values,
    labels={'x': 'Token', 'y': 'Activation Strength'},
    title='Brain Scan: Neuron 1612 (The "Double Agent")'
)

# Add markers so we can see the exact points
fig.update_traces(mode='lines+markers')

fig.show()

In [11]:
# The Mix: Technical Jargon + Bhagavad Gita 2.47
text_input = "The gradient descent algorithm minimizes loss by updating weights. You have a right to perform your prescribed duties, but you are not entitled to the fruits of your actions."


# 1. Run the model and catch the activations
logits, cache = model.run_with_cache(text_input)

# 2. Get the tokens as strings for our graph later
str_tokens = model.to_str_tokens(text_input)

In [12]:
for layer in range(0,12):
    print(f"Layer {layer}: {cache[f'blocks.{layer}.mlp.hook_post'].shape}")

Layer 0: torch.Size([1, 34, 3072])
Layer 1: torch.Size([1, 34, 3072])
Layer 2: torch.Size([1, 34, 3072])
Layer 3: torch.Size([1, 34, 3072])
Layer 4: torch.Size([1, 34, 3072])
Layer 5: torch.Size([1, 34, 3072])
Layer 6: torch.Size([1, 34, 3072])
Layer 7: torch.Size([1, 34, 3072])
Layer 8: torch.Size([1, 34, 3072])
Layer 9: torch.Size([1, 34, 3072])
Layer 10: torch.Size([1, 34, 3072])
Layer 11: torch.Size([1, 34, 3072])


In [13]:
acts = cache[f'blocks.{layer}.mlp.hook_post'][0]
valid_acts = acts[1:, :]

values,indices=torch.max(valid_acts,dim=0)

top_neuron_index=torch.argmax(values).item()
print(f"The new Champion Neuron is: {top_neuron_index}")

The new Champion Neuron is: 611


In [14]:


num_layers = 12

for layer in range(num_layers):
    # Shape: [batch, seq_len, d_mlp]
    acts = cache[f'blocks.{layer}.mlp.hook_post']
    print(f"Layer {layer}: {acts.shape}")

    # Take first batch element
    acts = acts[0]                     # [seq_len, d_mlp]

    # Remove BOS token (position 0)
    valid_acts = acts[1:, :]            # [seq_len-1, d_mlp]

    # Max over sequence dimension
    values, indices = torch.max(valid_acts, dim=0)  # [d_mlp]

    # Champion neuron for this layer
    top_neuron_index = torch.argmax(values).item()
    top_value = values[top_neuron_index].item()

    print(
        f"Layer {layer} → Champion Neuron: {top_neuron_index} "
        f"(activation={top_value:.4f})"
    )


Layer 0: torch.Size([1, 34, 3072])
Layer 0 → Champion Neuron: 1846 (activation=5.7334)
Layer 1: torch.Size([1, 34, 3072])
Layer 1 → Champion Neuron: 3020 (activation=4.3837)
Layer 2: torch.Size([1, 34, 3072])
Layer 2 → Champion Neuron: 352 (activation=3.7834)
Layer 3: torch.Size([1, 34, 3072])
Layer 3 → Champion Neuron: 1395 (activation=2.9592)
Layer 4: torch.Size([1, 34, 3072])
Layer 4 → Champion Neuron: 787 (activation=3.8430)
Layer 5: torch.Size([1, 34, 3072])
Layer 5 → Champion Neuron: 1888 (activation=3.2548)
Layer 6: torch.Size([1, 34, 3072])
Layer 6 → Champion Neuron: 1393 (activation=3.2327)
Layer 7: torch.Size([1, 34, 3072])
Layer 7 → Champion Neuron: 1360 (activation=3.4827)
Layer 8: torch.Size([1, 34, 3072])
Layer 8 → Champion Neuron: 1253 (activation=5.2777)
Layer 9: torch.Size([1, 34, 3072])
Layer 9 → Champion Neuron: 840 (activation=6.2107)
Layer 10: torch.Size([1, 34, 3072])
Layer 10 → Champion Neuron: 1793 (activation=5.2637)
Layer 11: torch.Size([1, 34, 3072])
Layer 11

In [21]:

num_layers = 12

for layer in range(num_layers):
    # ----- Find champion neuron -----
    acts = cache[f'blocks.{layer}.mlp.hook_post'][0]
    valid_acts = acts[1:, :]

    values, _ = torch.max(valid_acts, dim=0)
    champion_neuron = torch.argmax(values).item()
    champion_value = values[champion_neuron].item()

    print(
        f"Layer {layer} → Champion Neuron {champion_neuron} "
        f"(activation={champion_value:.4f})"
    )

    # ----- Plot activations across tokens -----
    neuron_data = acts[:, champion_neuron].detach().cpu().tolist()

    assert len(str_tokens) == len(neuron_data)

    fig = px.line(
        x=str_tokens,
        y=neuron_data,
        labels={'x': 'Token', 'y': 'Activation Strength'},
        title=f'Layer {layer} – Champion Neuron {champion_neuron}',
        markers=True
    )

    fig.show()

Layer 0 → Champion Neuron 1846 (activation=5.7334)


Layer 1 → Champion Neuron 3020 (activation=4.3837)


Layer 2 → Champion Neuron 352 (activation=3.7834)


Layer 3 → Champion Neuron 1395 (activation=2.9592)


Layer 4 → Champion Neuron 787 (activation=3.8430)


...

Layer 7 → Champion Neuron 1360 (activation=3.4827)


Layer 8 → Champion Neuron 1253 (activation=5.2777)


Layer 9 → Champion Neuron 840 (activation=6.2107)


Layer 10 → Champion Neuron 1793 (activation=5.2637)


Layer 11 → Champion Neuron 611 (activation=9.3640)


In [24]:

# We'll choose Layer 11 as it was the last layer processed in the previous plots
layer_to_plot = 5

# Get activations for the chosen layer
acts_layer = cache[f'blocks.{layer_to_plot}.mlp.hook_post'][0]

# Remove the BOS token (position 0) as done previously
valid_acts_layer = acts_layer[1:, :]

# Find the maximum activation for each neuron across the sequence dimension
# This gives a [d_mlp] tensor where each element is the max activation of one neuron
max_activations_per_neuron = torch.max(valid_acts_layer, dim=0).values

# Convert to a list for plotting
y_values = max_activations_per_neuron.detach().cpu().tolist()

# Create x-axis labels: neuron indices
x_values = list(range(len(y_values)))

# Create the scatter plot (or line for many points)
fig = px.line(
    x=x_values,
    y=y_values,
    labels={'x': 'Neuron Index (d_mlp)', 'y': 'Maximum Activation Strength'},
    title=f'Layer {layer_to_plot} – Maximum Activation per Neuron across Sequence',
    hover_name=[f'Neuron {i}' for i in x_values]
)

fig.update_traces(mode='lines') # Use lines as there are many neurons

fig.show()

This graph visualizes the maximum activation value for each neuron (on the x-axis) within Layer 11. The y-axis represents the strength of that maximum activation. This allows you to see the distribution of activation magnitudes across all neurons in a specific layer, which can often resemble a 'U-shape' with many neurons having low or zero activation and fewer neurons having high activations.

In [25]:

num_layers = 12
layer_indices = []
champion_activations = []

for layer in range(num_layers):
    acts = cache[f'blocks.{layer}.mlp.hook_post'][0]
    valid_acts = acts[1:, :]

    values, _ = torch.max(valid_acts, dim=0)
    champion_value = torch.max(values).item()

    layer_indices.append(layer)
    champion_activations.append(champion_value)

# Create the line plot
fig = px.line(
    x=layer_indices,
    y=champion_activations,
    labels={'x': 'Layer', 'y': 'Champion Neuron Activation Strength'},
    title='Maximum Neuron Activation Across Layers',
    markers=True
)

fig.show()

This graph visualizes the maximum activation value found in each layer's MLP (after removing the BOS token). The X-axis represents the layer number (from 0 to 11), and the Y-axis shows the activation strength of the most active neuron within that specific layer. This helps in understanding how the maximum 'excitement' of a neuron changes as information flows through the model's depth.